In [20]:
import os
import random
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import cv2
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
# from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers  import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#calculating area of damage

In [17]:
pred_labelled_data_dir = '/content/drive/MyDrive/CarDamage/Predictions for labelled data'

In [21]:
img_list=[]
for fn in glob.glob("/content/drive/MyDrive/CarDamage/Predictions for labelled data/*"):
        img_list.append(cv2.imread(fn, 0))

In [24]:
img_list

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
 

In [22]:
#Counting the damaged area
l3=[]
l4=[]
l5=[]
df=pd.DataFrame()
for i,j in zip(os.listdir(pred_labelled_data_dir),img_list):
   l3.append(i)
   l4.append(cv2.countNonZero(j))
   height, width = j.shape
   n_total = height * width
   l5.append(n_total)
df['Filename']=l3 
df['Count of damaged pixels']=l4
df['Total image size']=l5
df['Percentage of damaged pixels']=df['Count of damaged pixels']/df['Total image size']

In [23]:
df.head()

,Filename,Count of damaged pixels,Total image size,Percentage of damaged pixels
0,image2.jpg,17403,120320,0.144639
1,image1.jpeg,20286,141312,0.143555
2,image3.jpg,39647,174080,0.227752
3,image7.jpg,16231,196608,0.082555
4,image4.jpg,9157,196608,0.046575


In [25]:
df.shape

(48, 4)

In [27]:
labelled_data=pd.read_csv("/content/drive/MyDrive/CarDamage/Sev_cost_labelled.csv")

In [28]:
labelled_data.head()

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,image1.jpeg,4794,{},1,0,"{""name"":""polygon"",""all_points_x"":[80,104,178,2...","{""Impact Area"":""Front"",""Severity"":""Minor"",""Rep..."
1,image2.jpg,70052,{},1,0,"{""name"":""polygon"",""all_points_x"":[136,453,649,...","{""Severity"":""Minor"",""Impact Area"":""Side"",""Repa..."
2,image3.jpg,79882,{},1,0,"{""name"":""polygon"",""all_points_x"":[88,59,92,135...","{""Severity"":""Minor"",""Impact Area"":""Side"",""Repa..."
3,image4.jpg,26872,{},1,0,"{""name"":""polygon"",""all_points_x"":[68,124,160,1...","{""Severity"":""Minor"",""Impact Area"":""Side"",""Repa..."
4,image5.jpeg,8689,{},1,0,"{""name"":""polygon"",""all_points_x"":[76,112,137,1...","{""Severity"":""Minor"",""Impact Area"":""Front"",""Rep..."


In [29]:
import json
def string_to_dict(dict_string):
    # Convert to proper json format
    dict_string = dict_string.replace("'", '"').replace('u"', '"')
    return json.loads(dict_string)

In [30]:
bbox = (labelled_data["region_attributes"])

In [31]:
bbox = bbox.apply(string_to_dict)

In [32]:
bbox_list = list(bbox)

In [33]:
bbox_df = pd.DataFrame(bbox_list)

In [34]:
bbox_df.head(5)

,Impact Area,Severity,Repair Cost
0,Front,Minor,300
1,Side,Minor,500
2,Side,Minor,400
3,Side,Minor,250
4,Front,Minor,150


In [35]:
bbox_df.shape

(48, 3)

In [36]:
filenames = labelled_data['filename']

In [37]:
bbox_df['Filename']=filenames

In [38]:
final_df=df.merge(bbox_df,on='Filename',how='inner')

In [39]:
final_df.shape

(46, 7)

In [40]:
final_df.head()

,Filename,Count of damaged pixels,Total image size,Percentage of damaged pixels,Impact Area,Severity,Repair Cost
0,image2.jpg,17403,120320,0.144639,Side,Minor,500
1,image1.jpeg,20286,141312,0.143555,Front,Minor,300
2,image3.jpg,39647,174080,0.227752,Side,Minor,400
3,image7.jpg,16231,196608,0.082555,Side,Minor,220
4,image4.jpg,9157,196608,0.046575,Side,Minor,250


In [41]:
final_df['Repair Cost']=final_df['Repair Cost'].astype(int)

In [42]:
final_df.drop('Filename',axis=1,inplace=True)

In [43]:
cont=final_df[final_df.describe().columns]

In [44]:
cat=final_df.drop(final_df.describe().columns,axis=1)
from sklearn import preprocessing
dummy= pd.get_dummies(cat)
y=cont['Repair Cost']
cont.drop('Repair Cost',axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [45]:
fin_df=pd.concat([dummy , cont], axis=1)

In [46]:
fin_df.head()

,Impact Area_All around,Impact Area_Front,Impact Area_Moderate,Impact Area_Rear,Impact Area_Side,Severity_High,Severity_Minor,Severity_Moderate,Count of damaged pixels,Total image size,Percentage of damaged pixels
0,0,0,0,0,1,0,1,0,17403,120320,0.144639
1,0,1,0,0,0,0,1,0,20286,141312,0.143555
2,0,0,0,0,1,0,1,0,39647,174080,0.227752
3,0,0,0,0,1,0,1,0,16231,196608,0.082555
4,0,0,0,0,1,0,1,0,9157,196608,0.046575


In [47]:
fin_df.drop(['Count of damaged pixels',	'Total image size'],axis=1,inplace=True)

In [48]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(fin_df, y, test_size=0.2, random_state=0)
from sklearn import linear_model
regr = linear_model.LinearRegression()
# Train the model using the training sets
regr.fit(x_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(x_test)
from sklearn.metrics import mean_squared_error, r2_score
r2_score(y_test,y_pred)

-4.318561555635369

In [49]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(y_test, y_pred)

0.6456507587922459

In [50]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor()
reg.fit(x_train, y_train)

y_pred=reg.predict(x_test)
reg.score(x_test, y_test)

-8.211815657826447

In [51]:
import pickle

# save
with open('regr_model.pkl','wb') as f:
    pickle.dump(reg,f)